In [2]:
from torch.nn.modules.activation import LeakyReLU
import os
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pickle
import math
from itertools import chain
from torch.autograd import Variable

from tqdm import tqdm

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print("Device:", device)

# train_dir_color = '/scratch/mkolpe2s/MT/Main_data_folder/Segmentation_dataset/scannet/experiment_data/data/color_train/'
# train_dir_label = '/scratch/mkolpe2s/MT/Main_data_folder/Segmentation_dataset/scannet/experiment_data/data/label_train_n/'
# val_dir_color = '/scratch/mkolpe2s/MT/Main_data_folder/Segmentation_dataset/scannet/experiment_data/data/color_valid/'
# val_dir_label = '/scratch/mkolpe2s/MT/Main_data_folder/Segmentation_dataset/scannet/experiment_data/data/label_valid_n/'

train_dir_color = '/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/color_train/'
# train_dir_label = '/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_train/'
train_dir_label = '/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_train_n'
val_dir_color = '/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/color_valid/'
# val_dir_label = '/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_valid/'
val_dir_label = '/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_valid_n'
pos_dir = '/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/pose'

train_fns_color = os.listdir(train_dir_color)
train_fns_label = os.listdir(train_dir_label)
val_fns_color = os.listdir(val_dir_color)
val_fns_label = os.listdir(val_dir_label)
pose_fns = os.listdir(pos_dir)

print("train_fns_color:", len(train_fns_color), "train_fns_label:",len(train_fns_label),"val_fns_color:", len(val_fns_color),"val_fns_label:", len(val_fns_label))

class scannet(Dataset):
    
    def __init__(self, image_dir, label_dir, pose_dir):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.pose_dir = pose_dir
        self.image_fns = os.listdir(image_dir)
        self.label_fns = os.listdir(label_dir)
        self.pose_fns = os.listdir(pose_dir)

    def __len__(self):
        return len(self.image_fns)
    
    def __getitem__(self, index):
        image_fn = self.image_fns[index]
        # image_fn2 = self.image_fns[index+1]
        label_fn = self.label_fns[index]
        # pose_fn = self.pose_fns[index]
        image_fp = os.path.join(self.image_dir, image_fn)
        image = Image.open(image_fp).convert('RGB')
        image = np.array(image)
        label_fp = os.path.join(self.label_dir, image_fn.split('.')[0]+'.png')
        label = Image.open(label_fp)
        label = np.array(label)
        cityscape, label = image, label
        cityscape = self.transform(cityscape)
        label_class = torch.Tensor(label).long()

        pose_fp1 =  os.path.join(self.pose_dir, image_fn.split('.')[0]+'.txt')
        # pose_fp2 =  os.path.join(self.pose_dir, image_fn2.split('.')[0]+'.txt')
        # D = self.pose_process(pose_fp1,pose_fp2)
        D = self.test()
        return cityscape, label_class, D
    
    def test(self):

        gt_poses = []
        with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/pose.txt') as f:
            for l in f.readlines():
                l = l.strip('\n')
                gt_poses.append(np.array(l.split(' ')).astype(np.float32).reshape(4, 4))
        poses = [gt_poses[0], gt_poses[1]]
        D = torch.from_numpy(np.expand_dims(genDistM(poses), 0))
        return D

    def pose_process(self, file1, file2):


        poses = [gt_poses[0], gt_poses[1]]
        D = torch.from_numpy(np.expand_dims(genDistM(poses), 0))
    
    def pose_distance(p1, p2):
        rel_pose = np.dot(np.linalg.inv(p1), p2)
        R = rel_pose[:3, :3]
        t = rel_pose[:3, 3]

        return round(np.sqrt(np.linalg.norm(t) ** 2 + 2 * (1 - min(3.0, np.matrix.trace(R)) / 3)), 4)
    
    def genDistM(poses):
        n = len(poses)
        D = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                D[i, j] = pose_distance(poses[i], poses[j])
        return D

    def split_image(self, image):
        image = np.array(image)
        cityscape, label = image[:, :256, :], image[:, 256:, :]
        return cityscape, label
    
    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)

dataset = scannet(train_dir_color, train_dir_label, pos_dir)
print("Length of dataset:", len(dataset))

gt_poses = []
with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/pose.txt') as f:
    for l in f.readlines():
        l = l.strip('\n')
        gt_poses.append(np.array(l.split(' ')).astype(np.float32).reshape(4, 4))

def pose_distance(p1, p2):
    rel_pose = np.dot(np.linalg.inv(p1), p2)
    R = rel_pose[:3, :3]
    t = rel_pose[:3, 3]

    return round(np.sqrt(np.linalg.norm(t) ** 2 + 2 * (1 - min(3.0, np.matrix.trace(R)) / 3)), 4)
    
def genDistM(poses):
    n = len(poses)
    D = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            D[i, j] = pose_distance(poses[i], poses[j])
    return D

poses = [gt_poses[0], gt_poses[1]]
D = torch.from_numpy(np.expand_dims(genDistM(poses), 0))

tensor_list = []

class encoder(nn.Module):

    def __init__(self, num_classes):
        super(encoder, self).__init__()
        self.num_classes = num_classes
        self.contracting_11 = self.conv_block(in_channels=3, out_channels=64)
        self.contracting_12 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_21 = self.conv_block(in_channels=64, out_channels=128)
        self.contracting_22 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_31 = self.conv_block(in_channels=128, out_channels=256)
        self.contracting_32 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_41 = self.conv_block(in_channels=256, out_channels=512)
        self.contracting_42 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.middle = self.conv_block(in_channels=512, out_channels=1024)
      
    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels),
                                    nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels))
        return block
    
    def forward(self, X):
        contracting_11_out = self.contracting_11(X) # [-1, 64, 256, 256]
        contracting_12_out = self.contracting_12(contracting_11_out) # [-1, 64, 128, 128]
        contracting_21_out = self.contracting_21(contracting_12_out) # [-1, 128, 128, 128]
        contracting_22_out = self.contracting_22(contracting_21_out) # [-1, 128, 64, 64]
        contracting_31_out = self.contracting_31(contracting_22_out) # [-1, 256, 64, 64]
        contracting_32_out = self.contracting_32(contracting_31_out) # [-1, 256, 32, 32]
        contracting_41_out = self.contracting_41(contracting_32_out) # [-1, 512, 32, 32]
        contracting_42_out = self.contracting_42(contracting_41_out) # [-1, 512, 16, 16]
        middle_out = self.middle(contracting_42_out) # [-1, 1024, 16, 16]
        return middle_out, contracting_41_out, contracting_31_out, contracting_21_out, contracting_11_out

class GPlayer(nn.Module):


    def __init__(self):
        super(GPlayer, self).__init__()

        self.gamma2 = nn.Parameter(torch.randn(1), requires_grad=True).float()
        self.ell = nn.Parameter(torch.randn(1), requires_grad=True).float()
        self.sigma2 = nn.Parameter(torch.randn(1), requires_grad=True).float()


    def forward(self, D, Y):
        '''
        :param D: Distance matrix
        :param Y: Stacked outputs from encoder
        :return: Z: transformed latent space
        '''
        b=1
        l,c,h,w = Y.size()
        # print("----------------")
        # print("Y", Y.shape)
        # print("b", b)
        # print("l", l)
        # print("c", c)
        # print("h", h)
        # print("w", w)

        Y = Y.view(l,-1).cpu().float()
        # print("Y.shape",Y.shape)
        D = D.float()
        K = torch.exp(self.gamma2) * (1 + math.sqrt(3) * D / torch.exp(self.ell)) * torch.exp(-math.sqrt(3) * D / torch.exp(self.ell))
        I = torch.eye(l).expand(l, l).float()
        X,_ = torch.solve(Y, K+torch.exp(self.sigma2)*I)
        Z = K.bmm(X)
        Z = F.relu(Z)

        # l = 1
        # a, b, c, d = Y.size()
        # Y = Y.view(1,-1).cpu().float()
        # D = D.float()
        # K = torch.exp(self.gamma2) * (1 + math.sqrt(3) * D / torch.exp(self.ell)) * torch.exp(-math.sqrt(3) * D / torch.exp(self.ell))
        # I = torch.eye(l).expand(l, l).float()
        # X,_ = torch.solve(Y, K+torch.exp(self.sigma2)*I)
        # Z = K.bmm(X)
        # Z = F.relu(Z)
        # middle_out = Z[:, 0].view(a, 1024, 15, 20).to(device)

        # b,l,c,h,w = Y.size()
        # Y = Y.view(b,l,-1).cpu().float()
        # D = D.float()

        # K = torch.exp(self.gamma2) * (1 + math.sqrt(3) * D / torch.exp(self.ell)) * torch.exp(-math.sqrt(3) * D / torch.exp(self.ell))
        # I = torch.eye(l).expand(b, l, l).float()

        # X,_ = torch.solve(Y, K+torch.exp(self.sigma2)*I)

        # Z = K.bmm(X)

        # Z = F.relu(Z)

        return middle_out

class decoder(nn.Module):

    def __init__(self, num_classes):
        super(decoder, self).__init__()
        self.num_classes = num_classes

        self.expansive_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_12 = self.conv_block(in_channels=1024, out_channels=512)
        self.expansive_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_22 = self.conv_block(in_channels=512, out_channels=256)
        self.expansive_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_32 = self.conv_block(in_channels=256, out_channels=128)
        self.expansive_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_42 = self.conv_block(in_channels=128, out_channels=64)
        self.output = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=3, stride=1, padding=1)
        
    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels),
                                    nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels))
        return block
    
    def forward(self, middle_out, contracting_41_out, contracting_31_out, contracting_21_out, contracting_11_out):
        expansive_11_out = self.expansive_11(middle_out) # [-1, 512, 32, 32]
        expansive_12_out = self.expansive_12(torch.cat((expansive_11_out, contracting_41_out), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        expansive_21_out = self.expansive_21(expansive_12_out) # [-1, 256, 64, 64]
        expansive_22_out = self.expansive_22(torch.cat((expansive_21_out, contracting_31_out), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        expansive_31_out = self.expansive_31(expansive_22_out) # [-1, 128, 128, 128]
        expansive_32_out = self.expansive_32(torch.cat((expansive_31_out, contracting_21_out), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        expansive_41_out = self.expansive_41(expansive_32_out) # [-1, 64, 256, 256]
        expansive_42_out = self.expansive_42(torch.cat((expansive_41_out, contracting_11_out), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        output_out = self.output(expansive_42_out) # [-1, num_classes, 256, 256]
        return output_out

# model = UNet(num_classes=num_classes)

num_classes = 41
batch_size = 16

epochs = 1
lr = 0.001

dataset = scannet(train_dir_color, train_dir_label, pos_dir)
data_loader = DataLoader(dataset, batch_size=batch_size)

with open('data_loader.pkl', 'wb') as fp:
    pickle.dump(data_loader, fp)

encoder = encoder(num_classes=num_classes).to(device)

gplayer = GPlayer().to(device)

decoder = decoder(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()

encoder = torch.nn.DataParallel(encoder)
decoder = torch.nn.DataParallel(decoder)

parameters = chain(encoder.parameters(), gplayer.parameters(), decoder.parameters())
optimizer = torch.optim.Adam(parameters, lr=lr)

step_losses = []
epoch_losses = []

for epoch in tqdm(range(epochs)):
    epoch_loss = 0
    for X, Y, D_dum in tqdm(data_loader, total=len(data_loader), leave=False):
        X, Y = X.to(device), Y.to(device)
                
        middle_out, contracting_41_out, contracting_31_out, contracting_21_out, contracting_11_out = encoder(X)

        # Y = torch.stack(middle_out, dim = 1).cpu()
        # print(middle_out.size())
        # with open('middle_out.pkl', 'wb') as fp:
        #       pickle.dump(middle_out, fp)
        # with open('D.pkl', 'wb') as fp:
        #       pickle.dump(D, fp)
        if len(X) == 16:
            value = 16
            with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/D.pkl', 'rb') as fp:
                  D = pickle.load(fp)
        if len(X) == 6:
            value = 6
            with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/D_6.pkl', 'rb') as fp:
                  D = pickle.load(fp)
        Z = gplayer(D,middle_out)

        l,c,h,w = Z.size()

        # Y_pred_n = decoder(Z.to(device), contracting_41_out, contracting_31_out, contracting_21_out, contracting_11_out)

        # with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/Z.pkl', 'wb') as fp:
        #       pickle.dump(Z, fp)
        
        # with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/contracting_41_out.pkl', 'wb') as fp:
        #       pickle.dump(contracting_41_out, fp)
        
        # with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/contracting_31_out.pkl', 'wb') as fp:
        #       pickle.dump(contracting_31_out, fp)
        
        # with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/contracting_21_out.pkl', 'wb') as fp:
        #       pickle.dump(contracting_21_out, fp)
        
        # with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/contracting_11_out.pkl', 'wb') as fp:
        #       pickle.dump(contracting_11_out, fp)
        
        temp_list = []
        # print(Z.shape)
        for i in range(value):
            Z_new = torch.from_numpy((np.expand_dims(Z[i].detach().numpy(), axis=0)))
            contracting_41_out_new = torch.from_numpy((np.expand_dims(contracting_41_out[i].detach().numpy(), axis=0)))
            contracting_31_out_new = torch.from_numpy((np.expand_dims(contracting_31_out[i].detach().numpy(), axis=0)))
            contracting_21_out_new = torch.from_numpy((np.expand_dims(contracting_21_out[i].detach().numpy(), axis=0)))
            contracting_11_out_new = torch.from_numpy((np.expand_dims(contracting_11_out[i].detach().numpy(), axis=0)))
            # print(contracting_41_out_new.shape, contracting_31_out_new.shape, contracting_21_out_new.shape, contracting_11_out_new.shape)
            Y_pred_n = decoder(Z_new.to(device), contracting_41_out_new, contracting_31_out_new, contracting_21_out_new, contracting_11_out_new)
            temp_list.append(np.squeeze(Y_pred_n.detach().numpy()))
            # print(Y_pred_n.detach().numpy().shape)

        optimizer.zero_grad()
        Y_pred = np.array(temp_list)
        # print(Y_pred.shape)
        loss = criterion(torch.from_numpy(Y_pred), Y)
        loss = Variable(loss, requires_grad=True)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        step_losses.append(loss.item())
        print("----------------------")
        print("epoch_loss:",epoch_loss)
        print("step_losses:",loss.item() )
    epoch_losses.append(epoch_loss/len(data_loader))

with open('epoch_losses.pkl', 'wb') as fp:
    pickle.dump(epoch_losses, fp)

with open('step_losses.pkl', 'wb') as fp:
    pickle.dump(step_losses, fp)

model_name = "/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/U-Net_encoder.pth"
torch.save(encoder.state_dict(), model_name)

model_name = "/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/U-Net_gplayer.pth"
torch.save(gplayer.state_dict(), model_name)

model_name = "/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/U-Net_decoder.pth"
torch.save(decoder.state_dict(), model_name)

class scannet(Dataset):
    
    def __init__(self, image_dir, label_dir):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_fns = os.listdir(image_dir)
        self.label_fns = os.listdir(label_dir)
        
    def __len__(self):
        return len(self.image_fns)
    
    def __getitem__(self, index):
        image_fn = self.image_fns[index]
        label_fn = self.label_fns[index]
        image_fp = os.path.join(self.image_dir, image_fn)
        image = Image.open(image_fp).convert('RGB')
        image = np.array(image)
        print(index)
        print(image_fp)
        label_fp = os.path.join(self.label_dir, image_fn.split('.')[0]+'.png')
        label = Image.open(label_fp)
        label = np.array(label)
        print(label_fp)
        cityscape, label = image, label
        cityscape = self.transform(cityscape)
        label_class = torch.Tensor(label).long()
        return cityscape, label_class, 
    
    def split_image(self, image):
        image = np.array(image)
        cityscape, label = image[:, :256, :], image[:, 256:, :]
        return cityscape, label
    
    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)

test_batch_size = 8
dataset = scannet(val_dir_color, val_dir_label)
data_loader = DataLoader(dataset, batch_size=test_batch_size)

X, Y = next(iter(data_loader))
X, Y = X.to(device), Y.to(device)
Y_pred = model(X)
Y_pred = torch.argmax(Y_pred, dim=1)

inverse_transform = transforms.Compose([
    transforms.Normalize((-0.485/0.229, -0.456/0.224, -0.406/0.225), (1/0.229, 1/0.224, 1/0.225))
])

X = X.cpu().detach().numpy()
Y = Y.cpu().detach().numpy()
Y_pred = Y_pred.cpu().detach().numpy()

print(X.shape, Y.shape, Y_pred.shape)

with open('X.pkl', 'wb') as fp:
    pickle.dump(X, fp)

with open('Y.pkl', 'wb') as fp:
    pickle.dump(Y, fp)

with open('Y_pred.pkl', 'wb') as fp:
    pickle.dump(Y_pred, fp)

Device: cpu
train_fns_color: 230 train_fns_label: 230 val_fns_color: 49 val_fns_label: 49
Length of dataset: 230


  7%|▋         | 1/15 [00:43<10:04, 43.14s/it]

----------------------
epoch_loss: 3.8607120513916016
step_losses: 3.8607120513916016



 13%|█▎        | 2/15 [01:22<08:54, 41.09s/it]

----------------------
epoch_loss: 7.744035243988037
step_losses: 3.8833231925964355


...

----------------------
epoch_loss: 54.56041884422302
step_losses: 3.8870315551757812



100%|██████████| 1/1 [09:18<00:00, 558.71s/it]

----------------------
epoch_loss: 58.39350199699402
step_losses: 3.833083152770996


0
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/color_valid/4740.jpg
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_valid_n/4740.png
1
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/color_valid/4720.jpg
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_valid_n/4720.png
2
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/color_valid/4700.jpg
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_valid_n/4700.png
3
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/color_valid/4680.jpg
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_valid_n/4680.png
4
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/color_valid/4780.jpg
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_valid_n/4780.png
5
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/color_valid/4600.jpg
/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/label_valid_n/4600.png
6
/c

NameError: ignored

In [3]:
class scannet(Dataset):
    
    def __init__(self, image_dir, label_dir):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.image_fns = os.listdir(image_dir)
        self.label_fns = os.listdir(label_dir)
        
    def __len__(self):
        return len(self.image_fns)
    
    def __getitem__(self, index):
        image_fn = self.image_fns[index]
        label_fn = self.label_fns[index]
        image_fp = os.path.join(self.image_dir, image_fn)
        image = Image.open(image_fp).convert('RGB')
        image = np.array(image)
        print(index)
        print(image_fp)
        label_fp = os.path.join(self.label_dir, image_fn.split('.')[0]+'.png')
        label = Image.open(label_fp)
        label = np.array(label)
        print(label_fp)
        cityscape, label = image, label
        cityscape = self.transform(cityscape)
        label_class = torch.Tensor(label).long()
        return cityscape, label_class, 
    
    def split_image(self, image):
        image = np.array(image)
        cityscape, label = image[:, :256, :], image[:, 256:, :]
        return cityscape, label
    
    def transform(self, image):
        transform_ops = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)

test_batch_size = 6
dataset = scannet(val_dir_color, val_dir_label)
data_loader = DataLoader(dataset, batch_size=test_batch_size)

model_path_encoder = "/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/U-Net_encoder.pth"
model_path_gplayer = "/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/U-Net_gplayer.pth"
model_path_decoder = "/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/U-Net_decoder.pth"

encoder = encoder(num_classes=num_classes).to(device)
encoder.load_state_dict(torch.load(model_path_encoder))

gplayer = GPlayer().to(device)
gplayer.load_state_dict(torch.load(model_path_gplayer))

decoder = decoder(num_classes=num_classes).to(device)
decoder.load_state_dict(torch.load(model_path_decoder))

X, Y = next(iter(data_loader))

X, Y = X.to(device), Y.to(device)

middle_out, contracting_41_out, contracting_31_out, contracting_21_out, contracting_11_out = encoder(X)

if len(X) == 16:
    value = 16
    with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/D.pkl', 'rb') as fp:
          D = pickle.load(fp)
if len(X) == 6:
    value = 6
    with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/D_6.pkl', 'rb') as fp:
          D = pickle.load(fp)
Z = gplayer(D,middle_out)

l,c,h,w = Z.size()

temp_list = []
# print(Z.shape)
for i in range(value):
    Z_new = torch.from_numpy((np.expand_dims(Z[i].detach().numpy(), axis=0)))
    contracting_41_out_new = torch.from_numpy((np.expand_dims(contracting_41_out[i].detach().numpy(), axis=0)))
    contracting_31_out_new = torch.from_numpy((np.expand_dims(contracting_31_out[i].detach().numpy(), axis=0)))
    contracting_21_out_new = torch.from_numpy((np.expand_dims(contracting_21_out[i].detach().numpy(), axis=0)))
    contracting_11_out_new = torch.from_numpy((np.expand_dims(contracting_11_out[i].detach().numpy(), axis=0)))
    # print(contracting_41_out_new.shape, contracting_31_out_new.shape, contracting_21_out_new.shape, contracting_11_out_new.shape)
    Y_pred_n = decoder(Z_new.to(device), contracting_41_out_new, contracting_31_out_new, contracting_21_out_new, contracting_11_out_new)
    temp_list.append(np.squeeze(Y_pred_n.detach().numpy()))

Y_pred = np.array(temp_list)
Y_pred = torch.from_numpy(Y_pred)
Y_pred = torch.argmax(Y_pred, dim=1)

inverse_transform = transforms.Compose([
    transforms.Normalize((-0.485/0.229, -0.456/0.224, -0.406/0.225), (1/0.229, 1/0.224, 1/0.225))
])

X = X.cpu().detach().numpy()
Y = Y.cpu().detach().numpy()
Y_pred = Y_pred.cpu().detach().numpy()

print(X.shape, Y.shape, Y_pred.shape)

TypeError: ignored

In [1]:
with open('4580.txt') as f:
  for l in f.readlines():
        l = l.strip('\n')
        print(l)
      # gt_poses.append(np.array(l.split(' ')).astype(np.float32).reshape(4, 4))

0.851565 0.055141 0.521343 5.209820
-0.523823 0.049350 0.850397 6.443484
0.021163 -0.997258 0.070909 1.636609
0.000000 0.000000 0.000000 1.000000


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].plot(step_losses)
axes[1].plot(epoch_losses)

In [ ]:
test_batch_size = 8
inverse_transform = transforms.Compose([
    transforms.Normalize((-0.485/0.229, -0.456/0.224, -0.406/0.225), (1/0.229, 1/0.224, 1/0.225))
])

fig, axes = plt.subplots(test_batch_size, 3, figsize=(3*5, test_batch_size*5))

for i in range(test_batch_size):
    
    landscape = inverse_transform(torch.from_numpy(X[i])).permute(1, 2, 0).cpu().detach().numpy()
    label_class = Y[i]
    label_class_predicted = Y_pred[i]
    
    axes[i, 0].imshow(landscape)
    axes[i, 0].set_title("Landscape")
    axes[i, 1].imshow(label_class)
    axes[i, 1].set_title("Label Class")
    axes[i, 2].imshow(label_class_predicted)
    axes[i, 2].set_title("Label Class - Predicted")

In [ ]:
gt_poses = []
        with open('/content/drive/MyDrive/Master_thesis/Unet/unet_scannet/data/pose.txt') as f:
            for l in f.readlines():
                l = l.strip('\n')
                gt_poses.append(np.array(l.split(' ')).astype(np.float32).reshape(4, 4))

        def pose_distance(p1, p2):
            rel_pose = np.dot(np.linalg.inv(p1), p2)
            R = rel_pose[:3, :3]
            t = rel_pose[:3, 3]

            return round(np.sqrt(np.linalg.norm(t) ** 2 + 2 * (1 - min(3.0, np.matrix.trace(R)) / 3)), 4)
            
        def genDistM(poses):
            n = len(poses)
            D = np.zeros((n, n))
            for i in range(n):
                for j in range(n):
                    D[i, j] = pose_distance(poses[i], poses[j])
            return D

        poses = [gt_poses[0], gt_poses[1]]
        D = torch.from_numpy(np.expand_dims(genDistM(poses), 0))

        D = torch.from_numpy(np.expand_dims(D, 0))
        l = 1
        
        gamma2 = nn.Parameter(torch.randn(1), requires_grad=True).float()
        ell = nn.Parameter(torch.randn(1), requires_grad=True).float()
        sigma2 = nn.Parameter(torch.randn(1), requires_grad=True).float()

        print(middle_out.shape)

        a, b, c, d = middle_out.size()
        print(a, b, c, d)
        Y = middle_out.view(1,-1).cpu().float()
        print(Y.shape)
        D = D.float()
        K = torch.exp(gamma2) * (1 + math.sqrt(3) * D / torch.exp(ell)) * torch.exp(-math.sqrt(3) * D / torch.exp(ell))
        I = torch.eye(l).expand(l, l).float()
        X,_ = torch.solve(Y, K+torch.exp(sigma2)*I)
        Z = K.bmm(X)
        Z = F.relu(Z)
        middle_out = Z[:, 0].view(a, 1024, 15, 20).to(device)